# Простое введение в машинное обучение для тестировщиков
# Jupyter Notebook с пошаговыми объяснениями

## Шаг 1. Устанавливаем на компьютер нужные библиотеки для работы

In [1]:
pip install scikit-learn pandas numpy matplotlib seaborn jupyter


[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Шаг 2: Подключаем необходимые библиотеки

In [2]:
# Импортируем библиотеки - это как подключение инструментов для работы
import pandas as pd  # Для работы с таблицами данных
import numpy as np   # Для математических операций
from sklearn.datasets import load_breast_cancer  # Готовый датасет для примера
from sklearn.model_selection import train_test_split  # Функция для разделения данных
from sklearn.ensemble import RandomForestClassifier  # Наша модель машинного обучения
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Метрики для оценки

## Шаг 3: Загружаем датасет

In [3]:
# Загружаем датасет рака молочной железы
# Это готовый набор данных, который часто используют для обучения
cancer_data = load_breast_cancer()

# Показывает, какие "ключи" есть внутри объекта
print(cancer_data.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [4]:
# Получаем данные (признаки) и ответы (диагнозы)
X = cancer_data.data     # X - это признаки (размер опухоли, форма клеток и т.д.)
y = cancer_data.target   # y - это ответы (0 = злокачественная, 1 = доброкачественная)

print("📊 Информация о датасете:")
print(f"Количество пациентов: {X.shape[0]}")
print(f"Количество признаков: {X.shape[1]}")
print(f"Названия признаков: {cancer_data.feature_names}")
print(f"Названия классов: злокачественная, доброкачественная {cancer_data.target_names}")

📊 Информация о датасете:
Количество пациентов: 569
Количество признаков: 30
Названия признаков: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Названия классов: злокачественная, доброкачественная ['malignant' 'benign']


In [5]:
# Посмотрим, сколько каких диагнозов
# Функция np.unique возвращает уникальные значения массива y (например, классы в задаче классификации)
# Параметр return_counts=True дополнительно возвращает количество вхождений каждого уникального значения
unique, counts = np.unique(y, return_counts=True)

# Перебираем одновременно индексы (i) и пары (class_val, count), 
# где class_val — это уникальное значение класса, а count — количество элементов этого класса в y
for i, (class_val, count) in enumerate(zip(unique, counts)):
    class_name = cancer_data.target_names[i]

    # Выводим название класса и количество случаев в формате:
    #   "имя класса: число случаев"
    print(f"  {class_name}: {count} случаев")

  malignant: 212 случаев
  benign: 357 случаев


### Что происходит:

#### Мы загружаем готовый набор данных о раке молочной железы
#### X содержит измерения (как размер клеток, их форма и т.д.)
#### y содержит правильные ответы (злокачественная или доброкачественная опухоль)
#### Это как база данных, где для каждого пациента есть анализы и правильный диагноз

## Шаг 4: Разделяем данные на обучение и тестирование

In [6]:
# Разделяем данные на 2 части:
# 80% - для обучения модели (train)
# 20% - для проверки качества (test)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,                    # Наши данные
    test_size=0.2,          # 20% данных оставляем для тестирования
    random_state=42         # Фиксируем случайность для повторяемости результатов
)

# После разбиения у нас появляются 4 отдельных набора данных:

# 1. X_train — признаки для обучения (80% строк из X)
#    Используется моделью для поиска закономерностей между признаками и диагнозами.

# 2. y_train — правильные ответы для обучения (80% строк из y)
#    Это целевые метки, которые соответствуют X_train и нужны для того,
#    чтобы модель знала "правду" во время тренировки.

# 3. X_test — признаки для проверки (20% строк из X, которых модель не видела во время обучения)
#    Нужны для того, чтобы проверить модель на новых пациентах.

# 4. y_test — правильные ответы для X_test (20% строк из y)
#    Используются только для оценки качества: сравниваем предсказания модели (y_pred) с этими реальными диагнозами.

print("📊 Разделение данных:")
print(f"Для обучения: {X_train.shape[0]} пациентов")
print(f"Для тестирования: {X_test.shape[0]} пациентов")

📊 Разделение данных:
Для обучения: 455 пациентов
Для тестирования: 114 пациентов


In [7]:
# Проверим распределение диагнозов в тестовой выборке
unique_test, counts_test = np.unique(y_test, return_counts=True)
print("В тестовой выборке:")
for i, (class_val, count) in enumerate(zip(unique_test, counts_test)):
    class_name = cancer_data.target_names[i]
    print(f"  {class_name}: {count} случаев")

В тестовой выборке:
  malignant: 43 случаев
  benign: 71 случаев


### Что происходит:

#### Мы делим наши данные на две части
#### Одну часть покажем модели для обучения (как учебник)
#### Другую часть спрячем и будем использовать для проверки (как экзамен)
#### Это важно - модель не должна "видеть" правильные ответы на экзамене!

## Шаг 5: Обучаем модель

In [8]:
# Создаем модель машинного обучения
# RandomForest - это алгоритм, который создает много "деревьев решений"
model = RandomForestClassifier(
    n_estimators=100,    # Количество деревьев в "лесу"
    random_state=42      # Для повторяемости результатов
)

print("🤖 Начинаем обучение модели...")

🤖 Начинаем обучение модели...


In [9]:
# Обучаем модель на тренировочных данных
# Модель изучает связь между признаками (X_train) и диагнозами (y_train)
# Метод .fit() — это главный шаг обучения (тренировки) в машинном обучении.
model.fit(X_train, y_train)

print("✅ Модель обучена!")
print("Теперь модель 'знает', как признаки связаны с диагнозом")

✅ Модель обучена!
Теперь модель 'знает', как признаки связаны с диагнозом


### Что происходит при обучении:

#### Модель смотрит на признаки пациента (размер клеток, их форма и т.д.)
#### Смотрит на правильный диагноз для этого пациента
#### Ищет закономерности: "Если клетки такого размера И такой формы, то вероятно это злокачественная опухоль"
#### Повторяет это для всех пациентов из обучающей выборки
#### Создает "правила" для постановки диагноза

## Шаг 6: Делаем предсказания

In [10]:
# Теперь просим модель поставить диагноз пациентам из тестовой выборки
# В тестовой выборке (X_test) содержатся признаки тех пациентов, которых модель раньше не "видела".
# Модель НЕ знает правильные ответы для этих пациентов
y_pred = model.predict(X_test)

print("🔮 Модель сделала предсказания для тестовых пациентов")
# len(y_pred) должен совпадать с количеством строк в X_test
print(f"Количество данных: {len(X_test)}")
print(f"Количество предсказаний: {len(y_pred)}")

# Посмотрим на первые 10 предсказаний
print("\nПервые 10 предсказаний:")
print("Правильный диагноз → Предсказание модели")

# Цикл по первым 10 пациентам из тестовой выборки
for i in range(10):
    # Берём правильное название диагноза (0 или 1 заменяем на "malignant"/"benign")
    true_name = cancer_data.target_names[y_test[i]]
    # Берём предсказанное моделью название диагноза
    pred_name = cancer_data.target_names[y_pred[i]]
    # Сравниваем предсказание с правильным ответом.
    status = "✅" if y_test[i] == y_pred[i] else "❌"
    print(f"{true_name} → {pred_name} {status}")

🔮 Модель сделала предсказания для тестовых пациентов
Количество данных: 114
Количество предсказаний: 114

Первые 10 предсказаний:
Правильный диагноз → Предсказание модели
benign → benign ✅
malignant → malignant ✅
malignant → malignant ✅
benign → benign ✅
benign → benign ✅
malignant → malignant ✅
malignant → malignant ✅
malignant → malignant ✅
benign → malignant ❌
benign → benign ✅


### Что происходит:

#### Модель смотрит на данные пациентов, которых она НЕ видела при обучении
#### Применяет выученные правила и делает предсказания
#### Мы сравниваем предсказания с правильными ответами

## Шаг 7: Оцениваем качество модели

In [11]:
# Считаем основные метрики качества
accuracy = accuracy_score(y_test, y_pred)      # Точность - доля правильных ответов
precision = precision_score(y_test, y_pred)    # Precision - из предсказанных "доброкачественных", сколько действительно доброкачественных
recall = recall_score(y_test, y_pred)          # Recall - из всех доброкачественных, сколько мы нашли
f1 = f1_score(y_test, y_pred)                  # F1 - среднее между precision и recall

print("📊 РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ МОДЕЛИ:")
print("="*50)
print(f"Accuracy (Точность):  {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"Precision (Точность положительных): {precision:.3f} ({precision*100:.1f}%)")
print(f"Recall (Полнота):     {recall:.3f} ({recall*100:.1f}%)")
print(f"F1-Score (Общая оценка): {f1:.3f} ({f1*100:.1f}%)")

# Объясняем, что означают эти цифры
print("\n📝 Что это означает:")
print(f"• Из {len(y_test)} пациентов модель правильно диагностировала {int(accuracy * len(y_test))}")
print(f"• Это составляет {accuracy*100:.1f}% точности")

📊 РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ МОДЕЛИ:
Accuracy (Точность):  0.965 (96.5%)
Precision (Точность положительных): 0.959 (95.9%)
Recall (Полнота):     0.986 (98.6%)
F1-Score (Общая оценка): 0.972 (97.2%)

📝 Что это означает:
• Из 114 пациентов модель правильно диагностировала 110
• Это составляет 96.5% точности


## Шаг 7: Детальный анализ ошибок

In [12]:
from sklearn.metrics import confusion_matrix

In [13]:
# Создаем матрицу ошибок - таблицу, показывающую где модель ошиблась
cm = confusion_matrix(y_test, y_pred)

print("\n🔍 ДЕТАЛЬНЫЙ АНАЛИЗ ОШИБОК:")
print("="*40)


🔍 ДЕТАЛЬНЫЙ АНАЛИЗ ОШИБОК:


In [15]:
# Объясняем каждую ячейку матрицы ошибок
tn, fp, fn, tp = cm.ravel()  # True Negative, False Positive, False Negative, True Positive

print("Результаты по типам:")
print(f"✅ Правильно определили злокачественные: {tn}")
print(f"✅ Правильно определили доброкачественные: {tp}")
print(f"❌ Ложная тревога (сказали 'доброкачественная', а была 'злокачественная'): {fp}")
print(f"❌ Пропустили болезнь (сказали 'злокачественная', а была 'доброкачественная'): {fn}")

print(f"\nВсего ошибок: {fp + fn} из {len(y_test)} ({(fp + fn)/len(y_test)*100:.1f}%)")

print("\n⚕️ В медицинской диагностике:")
if fn > 0:
    print(f"ОПАСНО: {fp} раз пропустили злокачественную опухоль!")
    print("   (Пациент думает, что здоров, а болезнь прогрессирует)")

if fp > 0:
    print(f"СТРЕСС: {fn} раз ложно диагностировали злокачественную опухоль")
    print("   (Пациент зря переживает, нужны дополнительные анализы)")

Результаты по типам:
✅ Правильно определили злокачественные: 40
✅ Правильно определили доброкачественные: 70
❌ Ложная тревога (сказали 'доброкачественная', а была 'злокачественная'): 3
❌ Пропустили болезнь (сказали 'злокачественная', а была 'доброкачественная'): 1

Всего ошибок: 4 из 114 (3.5%)

⚕️ В медицинской диагностике:
ОПАСНО: 3 раз пропустили злокачественную опухоль!
   (Пациент думает, что здоров, а болезнь прогрессирует)
СТРЕСС: 1 раз ложно диагностировали злокачественную опухоль
   (Пациент зря переживает, нужны дополнительные анализы)
